In [1]:
ROBOFLOW_API_KEY = "6wzCHB3PAi6vCKlHrZT5"

from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("rm-2021").project("armor-plates")
dataset = project.version(26).download("coco")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Armor-Plates-26 in coco:: 100%|██████████| 5568/5568 [00:07<00:00, 705.08it/s]


In [87]:
from yolosutils import CocoDetection, get_collate_fn, get_yolos_feature_extractor
import torch
from pytorch_lightning import LightningModule
from transformers import AutoModelForObjectDetection
from torch.optim import AdamW
MODEL_SIZE = 'tiny'

feature_extractor = get_yolos_feature_extractor(MODEL_SIZE)

train_dataset = CocoDetection(img_folder=(dataset.location + '/train'), feature_extractor=feature_extractor)
val_dataset = CocoDetection(img_folder=(dataset.location + '/valid'), feature_extractor=feature_extractor,
                            train=False)

from torch.utils.data import Subset

indices = torch.arange(25)
val_dataset = Subset(val_dataset, indices)

from torch.utils.data import DataLoader

collate_fn = get_collate_fn(feature_extractor)

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, shuffle=True, batch_size=1, num_workers=0)
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=1, num_workers=0)

from torchmetrics.detection.mean_ap import MeanAveragePrecision

MODEL_PREFIX = 'hustvl/yolos-'

class YoloS(LightningModule):

    def __init__(self, lr, weight_decay, model_size):
        super().__init__()
        # replace COCO classification head with custom head
        self.model = AutoModelForObjectDetection.from_pretrained(MODEL_PREFIX + model_size,
                                                                 # num_labels=2,
                                                                 # ignore_mismatched_sizes=True
                                                                 )
        # see https://github.com/PyTorchLightning/pytorch-lightning/pull/1896
        self.lr = lr
        self.weight_decay = weight_decay
        self.stored_val_dataloader = val_dataloader
        self.stored_train_dataloader = train_dataloader
        self.batch_size = 1
        self.map = MeanAveragePrecision(box_format="cxcywh")
        self.save_hyperparameters()

    def on_train_start(self):
        self.log("batch_size", float(self.batch_size))

    def forward(self, pixel_values):
        outputs = self.model(pixel_values=pixel_values)
        return outputs

    def common_step(self, batch, batch_idx):
        pixel_values = batch["pixel_values"]
        labels = [{k: v.to(self.device) for k, v in t.items()} for t in batch["labels"]]

        outputs = self.model(pixel_values=pixel_values, labels=labels)

        loss = outputs.loss
        loss_dict = outputs.loss_dict

        return loss, loss_dict

    def training_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("train/loss",
                 loss)  # logging metrics with a forward slash will ensure the train and validation metrics as split into 2 separate sections in the W&B workspace
        for k, v in loss_dict.items():
            self.log("train/" + k,
                     v.item())  # logging metrics with a forward slash will ensure the train and validation metrics as split into 2 separate sections in the W&B workspace

        return loss

    def validation_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)
        self.log("validation/loss",
                 loss)
        for k, v in loss_dict.items():
            self.log("validation/" + k,
                     v.item())

        actualdata = batch['labels'][0]
        actualboxes = actualdata['boxes'].cpu().double()
        actualclasses = actualdata['class_labels'].cpu().double()
        modelout = self.model(pixel_values=batch['pixel_values'])
        pred_boxes = modelout.pred_boxes[0].cpu().double()

        pred_scores=modelout.logits.cpu().double()[0, :, :].softmax(-1).max(-1).values
        pred_labels=modelout.logits.cpu().double()[0, :, :].softmax(-1).max(-1).indices.float()

        notempty = torch.logical_or(pred_labels == 1, pred_labels == 2)
        pred_labels = pred_labels[notempty]
        pred_scores = pred_scores[notempty]
        pred_boxes = pred_boxes[notempty]

        preds = [
            dict(
                boxes=pred_boxes,
                scores=pred_scores,
                labels=pred_labels
            )
        ]
        target = [
            dict(
                boxes=actualboxes,
                labels=actualclasses,
            )
        ]
        errored = False
        try:
            testmap = MeanAveragePrecision(box_format="cxcywh")
            testmap.update(preds, target)
            testmap.compute()
            print(testmap.compute())
        except Exception:
            errored = True
        if not errored:
            self.map.update(preds, target)
            # print(self.map.compute()['map'])
        return loss

    def validation_epoch_end(self, validation_step_outputs):
        print(self.map.compute())

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.lr,
                                      weight_decay=self.weight_decay)
        return optimizer

    def val_dataloader(self):
        return self.stored_val_dataloader

    def train_dataloader(self):
        return self.stored_train_dataloader

loading annotations into memory...
Done (t=0.64s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [88]:
model = YoloS(lr=0, weight_decay=0, model_size=MODEL_SIZE)
model.load_from_checkpoint(checkpoint_path='../epoch=8-step=1098.ckpt', model_size=MODEL_SIZE)

from pytorch_lightning import Trainer

#We need to limit devices because parallelism screws everything up
trainer = Trainer(
    # fast_dev_run=True,
    devices=1,
    accelerator='cpu',
)

metrics = trainer.validate(model, verbose=True)
print(metrics)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.


Validation: 0it [00:00, ?it/s]

{'map': tensor(-1.), 'map_50': tensor(-1.), 'map_75': tensor(-1.), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(-1.), 'mar_1': tensor(-1.), 'mar_10': tensor(-1.), 'mar_100': tensor(-1.), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(-1.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.)}
{'map': tensor(0.), 'map_50': tensor(0.), 'map_75': tensor(0.), 'map_small': tensor(0.), 'map_medium': tensor(-1.), 'map_large': tensor(-1.), 'mar_1': tensor(0.), 'mar_10': tensor(0.), 'mar_100': tensor(0.), 'mar_small': tensor(0.), 'mar_medium': tensor(-1.), 'mar_large': tensor(-1.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.)}
{'map': tensor(0.), 'map_50': tensor(0.), 'map_75': tensor(0.), 'map_small': tensor(0.), 'map_medium': tensor(-1.), 'map_large': tensor(-1.), 'mar_1': tensor(0.), 'mar_10': tensor(0.), 'mar_100': tensor(0.), 'mar_small': tensor(0.), 'mar_medium': tensor(-1.), 'mar_large': tensor(-1.), 'map_p

In [80]:
iterdata = iter(val_dataloader)
for _ in range(2):
    next(iterdata)
batch = next(iterdata)

actualdata = batch['labels'][0]
actualboxes = actualdata['boxes'].cpu().double()
actualclasses = actualdata['class_labels'].cpu().double()
modelout = model(pixel_values=batch['pixel_values'])
pred_boxes = modelout.pred_boxes[0].cpu().double()



boxes=pred_boxes
scores=modelout.logits.cpu().double()[0, :, :].softmax(-1).max(-1).values
labels=modelout.logits.cpu().double()[0, :, :].softmax(-1).max(-1).indices

notempty = torch.logical_or(labels == 1, labels == 2)
labels = labels[notempty]
scores = scores[notempty]
boxes = boxes[notempty]
print(labels)
print(scores)
print(boxes)
print(actualclasses)
print(actualboxes)

tensor([1, 1, 1, 1, 1, 1])
tensor([0.7079, 0.7251, 0.4816, 0.5856, 0.5449, 0.7733], dtype=torch.float64,
       grad_fn=<IndexBackward0>)
tensor([[0.6719, 0.1988, 0.1555, 0.0944],
        [0.6200, 0.2183, 0.2650, 0.1404],
        [0.1481, 0.8249, 0.2956, 0.3485],
        [0.6107, 0.2154, 0.1867, 0.1280],
        [0.6564, 0.2253, 0.1241, 0.0711],
        [0.6348, 0.2207, 0.2038, 0.0964]], dtype=torch.float64,
       grad_fn=<IndexBackward0>)
tensor([1., 1.], dtype=torch.float64)
tensor([[0.5042, 0.4351, 0.1046, 0.1298],
        [0.2975, 0.4718, 0.0565, 0.1310]], dtype=torch.float64)
